In [1]:
import numpy as np
from my_stuff import BinanceUSDMTestKeys, ApexTestKeys, MufexTestKeys, BybitTestKeys,MufexKeys
from quantfreedom.helper_funcs import candles_to_df
from quantfreedom.exchanges.binance_exchange.binance_usdm import BinanceUSDM
from quantfreedom.exchanges.bybit_exchange.bybit import Bybit
from quantfreedom.enums import *
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.exchanges.apex_exchange.apex import Apex
from quantfreedom.simulate import run_df_backtest, or_backtest
from quantfreedom.strategies.strategy import Strategy
from quantfreedom.strategies.ex_strat_01 import RSIBelowAbove


%load_ext autoreload
%autoreload 2

In [2]:
mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

In [3]:
candles = mufex_main.get_candles(
    symbol="BTCUSDT",
    timeframe="5m",
    since_date_ms=1699988400000,
    until_date_ms=1700092800000,
)

candles_to_df(candles=candles)

,timestamp,open,high,low,close,volume
datetime,,,,,,
2023-11-14 19:00:00,1699988400000,35245.2,35305.7,35109.0,35180.6,30.490
2023-11-14 19:05:00,1699988700000,35180.6,35457.4,35180.6,35419.0,36.090
2023-11-14 19:10:00,1699989000000,35419.0,35453.9,35360.0,35393.5,13.999
2023-11-14 19:15:00,1699989300000,35393.5,35473.4,35360.0,35365.8,14.061
2023-11-14 19:20:00,1699989600000,35365.8,35392.8,35284.8,35333.9,14.049
...,...,...,...,...,...,...
2023-11-15 23:35:00,1700091300000,37822.7,37860.4,37785.4,37860.2,7.898
2023-11-15 23:40:00,1700091600000,37860.2,37896.0,37835.1,37864.7,7.153
2023-11-15 23:45:00,1700091900000,37864.7,37889.9,37804.1,37804.1,5.024


In [4]:


mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)

# backtest_settings = BacktestSettings(
#     array_size=200,
#     gains_pct_filter=-90,
#     total_trade_filter=10,
#     upside_filter=-np.inf,
# )
backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([5]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2.5]),
    sl_based_on_add_pct=np.array([0.05, 0.1, 0.15]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5]),
    trail_sl_when_pct=np.array([1]),
)
static_os = StaticOrderSettings(
    starting_bar=50,
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)
strategy = RSIBelowAbove(
    rsi_is_below=np.array([60]),
    rsi_length=np.array([14]),
    long_short="long",
)

In [5]:
run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=strategy,
    logger_bool=False,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 1
Total order settings to test: 3
Total combinations of settings to test: 3
Total candles: 348
Total candles to test: 1,044


,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,total_pnl,ending_eq
0,0,0,10.0,5,5,48.954,50.000,0.680,346.786,489.536,1489.536
1,0,1,7.0,5,2,91.328,71.429,0.907,227.818,913.280,1913.280
2,0,2,6.0,5,1,86.508,83.333,0.961,164.726,865.076,1865.076


In [6]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=strategy,
    dos_index=2,
    ind_set_index=0,
    plot_results=True,
    logger_bool=True,
)

In [7]:
order_records_df

,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,equity,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
0,0,2,55,1700004900000,2023-11-14 23:35:00,EntryFilled,1000.000,951.739,3628.846,48.261,...,3677.107,35636.2,NaN,0.103,3677.107,NaN,0.600,35409.4,2.2,36406.0
1,0,2,56,1700005200000,2023-11-14 23:40:00,EntryFilled,1000.000,851.897,11262.828,148.103,...,4104.978,35605.7,NaN,0.218,7782.085,NaN,0.600,35409.4,2.0,36355.4
2,0,2,57,1700005500000,2023-11-14 23:45:00,EntryFilled,1000.000,699.837,23108.866,300.163,...,4364.116,35580.2,NaN,0.341,12146.201,NaN,0.600,35409.4,2.0,36316.4
3,0,2,58,1700005800000,2023-11-14 23:50:00,EntryFilled,1000.000,495.717,39226.615,504.283,...,4475.831,35572.2,NaN,0.467,16622.032,NaN,0.500,35409.4,1.9,36290.4
4,0,2,87,1700014500000,2023-11-15 02:15:00,StopLossFilled,878.854,878.854,NaN,NaN,...,NaN,NaN,35409.4,NaN,NaN,-121.146,NaN,NaN,NaN,NaN
5,0,2,88,1700014800000,2023-11-15 02:20:00,EntryFilled,878.854,822.188,6055.366,56.666,...,6112.032,35421.8,NaN,0.173,6112.032,NaN,0.200,35334.8,1.2,35841.1
6,0,2,89,1700015100000,2023-11-15 02:25:00,EntryFilled,878.854,715.634,17088.124,163.220,...,5083.946,35452.4,NaN,0.316,11195.978,NaN,0.300,35334.8,1.3,35885.5
7,0,2,90,1700015400000,2023-11-15 02:30:00,EntryFilled,878.854,555.573,33356.164,323.281,...,5395.343,35450.8,NaN,0.468,16591.321,NaN,0.300,35334.8,1.3,35895.3
8,0,2,91,1700015700000,2023-11-15 02:35:00,EntryFilled,878.854,347.417,53935.484,531.437,...,4519.436,35480.7,NaN,0.595,21110.757,NaN,0.300,35334.8,1.4,35923.2
9,0,2,166,1700038200000,2023-11-15 08:50:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.005,35624.0,NaN,NaN


In [13]:
Mufex(
    api_key=MufexTestKeys.api_key,
    secret_key=MufexTestKeys.secret_key,
    use_test_net=True,
).create_order(
    symbol="BTCUSDT",
    position_mode=PositionModeType.BuySide,
    buy_sell="buy",
    order_type="market",
    asset_size=0.001,
    time_in_force="GoodTillCancel",
)

'42da5663-8ba3-4f26-958f-b1e58837ee82'

In [9]:
Apex(
    api_key=ApexTestKeys.api_key,
    secret_key=ApexTestKeys.secret_key,
    passphrase=ApexTestKeys.passphrase,
    stark_key_private=ApexTestKeys.stark_key_private,
    stark_key_public=ApexTestKeys.stark_key_public,
    stark_key_y=ApexTestKeys.stark_key_y,
    use_test_net=True,
).create_entry_market_order(
    symbol="BTC-USDC",
    buy_sell="sell",
    asset_size=0.005,
)

'518445551813919084'

In [14]:
BinanceUSDM(
    api_key=BinanceUSDMTestKeys.api_key,
    secret_key=BinanceUSDMTestKeys.secret_key,
    use_test_net=True,
).create_order(
    symbol="BTCUSDT",
    side="BUY",
    type="MARKET",
    quantity=0.005,
    timeInForce=None,
)

3544585474

In [15]:
Bybit(
    use_test_net=True,
    api_key=BybitTestKeys.api_key,
    secret_key=BybitTestKeys.secret_key,
).create_order(
    symbol="BTCUSDT",
    buy_sell="buy",
    position_mode=1,
    order_type="market",
    asset_size=0.01,
)

'cf93b055-e462-4cf7-b61e-7773d5971371'